# Simulation and plotting

In [ ]:
%matplotlib notebook
import math
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import h5py
import pickle
import torch
import torch.jit
from model.DEM import DeepEuler, AdaptiveDeepEuler
from model.Euler import Euler
from utils.scalers import StandardScaler

torch.set_default_dtype(torch.float64)
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 15.0})

In [ ]:
ro_L = 9.970639504998557e+02
P_inf = 1.0e+5
p_v = 3.166775638952003e+03
sigma = 0.071977583160056
R_E = 10.0/1.0e6
gam = 1.4
c_L = 1.497251785455527e+03
mu_L = 8.902125058209557e-04
theta = 0.0
P_A1 = 1e5
P_A2 = 0.0
f = 20e3 #20 kHz
f2 = 0.0

class Keller():
    '''The ODE function of the Keller-Miksis equation'''
    
    def __init__(self):
        super().__init__()
        
        C = np.zeros(13)
        twr = 1.0/(R_E*f)
        C[0] = (P_inf - p_v + 2*sigma/R_E)/ro_L*twr*twr
        C[1] = (1-3*gam)/(ro_L*c_L)*(P_inf - p_v + 2*sigma/R_E)*twr
        C[2] = (P_inf - p_v)/ro_L * twr*twr
        C[3] = 2*sigma/(ro_L*R_E) *twr*twr
        C[4] = 4*mu_L/(ro_L*R_E*R_E) / f
        C[5] = P_A1/ro_L * twr*twr
        C[6] = P_A2/ro_L *twr*twr
        C[7] = R_E * 2*math.pi*f * P_A1/(ro_L*c_L) * twr*twr
        C[8] = R_E * 2*math.pi*f * P_A2/(ro_L*c_L) * twr*twr
        C[9] = R_E*f / c_L
        C[10] = 3*gam
        C[11] = f2 / f
        C[12] = theta
        self.C = C
    
    def ode(self, t, x):
        '''ODE function'''
        C = self.C
        dxdt = np.ones(x.shape)
        rx0 = 1.0 / x[0];
        
        N = (C[0]+C[1]*x[1])*pow(rx0,C[10]) - C[2]*(1.0+C[9]*x[1]) -C[3]*rx0 -C[4]*x[1]*rx0 -\
        (1.5 - 0.5*C[9]*x[1])*x[1]*x[1] -\
        (C[5]*np.sin(2.0*math.pi*t) + C[6]*np.sin(2.0*math.pi*C[11]*t + C[12])) * (1.0+C[9]*x[1])-\
        x[0] * (C[7]*np.cos(2.0*math.pi*t) + C[8]*np.cos(2.0*math.pi*C[11]*t+C[12]) );
        
        D = x[0] - C[9]*x[0]*x[1] + C[4]*C[9];
                
        dxdt[0] = x[1]
        dxdt[1] = N / D
        return dxdt

## Integrate the ODE

In [ ]:
keller = Keller()
sol = scipy.integrate.solve_ivp(keller.ode, [0, 5], [ 1.0, 0.0], rtol=1e-10, atol=1e-10)

In [ ]:
sol2 = scipy.integrate.solve_ivp(keller.ode, [0, 1], [ 2.0, 200.0], rtol=1e-10, atol=1e-10)

In [ ]:
eul_sol = scipy.integrate.solve_ivp(keller.ode, [0, 3], [ 1.0, 0.0], method=Euler, h=1e-4)

Change the *traced_model_path* and *scaler_path* to point to your trained model and its scaler

In [ ]:
begin = datetime.now()
with open("training/out_scaler_keller_f_2211020939.psca.pickle", "rb") as file:
    out_scaler = pickle.load(file)
with open("training/scaler_keller_f_2211020939.psca",'r') as file:
    in_scaler = StandardScaler(file)
    
model = torch.jit.load("training/traced_model_keller_f_e198_2211020939.pt")
model.eval()

dem_sol = scipy.integrate.solve_ivp(keller.ode, [0, 3], [ 1.0, 0.0], method=DeepEuler, h=1e-5, 
                                    out_scaler=out_scaler, in_scaler = in_scaler, traced_model=model)
end = datetime.now()
print(end-begin)

In [ ]:
begin = datetime.now()
ad_dem_sol = scipy.integrate.solve_ivp(keller.ode, [0, 5], [ 1.0, 0], method=AdaptiveDeepEuler, 
                                    first_step=0.1, rtol=1e-5, atol=1e-5,
                                    traced_model_path="training/traced_model_keller_ok_e1519_2210121923.pt", 
                                    scaler_path="training/scaler_keller_ok_2210121923.psca")
end = datetime.now()
print(end-begin)

## Plots

### Accurate phase space (with DOPRI)

In [ ]:
fig = plt.figure(num="PhaseSpace")
ax = fig.add_subplot(projection="3d")
m = sol.t < 1
ax.plot(sol.t[m], sol.y[0,m],sol.y[1,m])
m = (sol.t >= 1) * (sol.t < 2)
ax.plot(np.mod(sol.t[m],1), sol.y[0,m],sol.y[1,m])
m = (sol.t >= 2) * (sol.t < 3)
ax.plot(np.mod(sol.t[m],1), sol.y[0,m],sol.y[1,m])
m = (sol.t >= 3) * (sol.t < 4)
ax.plot(np.mod(sol.t[m],1), sol.y[0,m],sol.y[1,m])
m = (sol.t >= 4) * (sol.t < 5)
ax.plot(np.mod(sol.t[m],1), sol.y[0,m],sol.y[1,m])
#plt.scatter(eul_sol.y[1,:],eul_sol.y[0,:], s=10, label="Euler")
#plt.scatter(dem_sol.y[1,:],dem_sol.y[0,:], s=10, label="DEM")
ax.set_xlabel("$t$")
ax.set_ylabel("$x_1$")
ax.set_zlabel("$x_2$")
#ax.legend()
plt.show()

In [ ]:
ax.azim = 150   # z rotation (default=270)
ax.elev = 42    # x rotation (default=0)
ax.dist = 5    # define perspective (default=10)
fig.canvas.draw()
#ax.set_xlim3d(low_x, high_x)
#ax.set_ylim3d(low_y, high_y)
#ax.set_zlim3d(low_z, high_z)

In [ ]:
plt.savefig("PhaseSpace3d.pdf")
import pickle
with open("PhaseSpace3d.fig.pickle", "wb") as file:
    pickle.dump(fig, file)

### Comparison

In [ ]:
plt.figure(num="Comparison")
plt.plot(sol.t,sol.y[0,:], color="black", label="Dopri")
plt.plot(eul_sol.t,eul_sol.y[0,:], color="silver", label="Euler 1e-4")
plt.plot(dem_sol.t,dem_sol.y[0,:], color="purple", label="DEM 1e-4", linestyle="--", dashes=(5,5))
#plt.plot(ad_dem_sol.t, ad_dem_sol.y[0,:], linestyle="--", dashes=(6,4), color="orange", label="ADEM")
plt.xlabel("$t$")
plt.ylabel("$x_1$")
plt.xlim([0,3])
#plt.ylim([0, 3])
plt.legend()
plt.show()

In [ ]:
plt.xlim([2, 3])

In [ ]:
plt.savefig("Keller_Dopri_Euler_2210121923.pdf")